In [1]:
#Import relevant libraries
import astropy
import numpy as np
import ccdproc
import os
import glob
import matplotlib.pyplot as plt
from astropy.nddata import CCDData
import scipy

In [2]:
#TASK 14 AND 15
#Here we are subtracting the master dark,bias and dividing by the masterflat from the second half of the master science V


masterflatV = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expflatV.fits', unit='adu')
masterbias = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expbias.fits', unit='adu')
masterdark = astropy.nddata.CCDData.read('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/expdark.fits', unit='adu')


#file_location = os.path.join('20191111/', 'science_B/', '*.fits')
#file_location = os.path.join('GlobularClusters/', 'science_B/', '*.fits')
filenames = glob.glob('C:/Users/steve/OneDrive - University of Birmingham/LH Observatory Laboratory/Glob Cluter Final Code/GlobClusters/20191111/science_V2/*.fits')

scienceimagesV = []

for file in filenames:
    scienceImg = astropy.nddata.CCDData.read(file, unit='adu')
    hdul = astropy.nddata.CCDData.to_hdu(scienceImg)
    scienceSub1 = scienceImg.subtract(masterbias)
    scienceSub2 = scienceSub1.subtract(masterdark)
    divscience = scienceSub2.divide(masterflatV)
    #divscience2 = divscience.divide(60) #to ensure exposure time is 1 sec
    scienceimagesV.append(divscience)
    
    
print(scienceimagesV)

INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]
INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


INFO: using the unit adu passed to the FITS reader instead of the unit adu in the FITS file. [astropy.nddata.ccddata]


[CCDData([[ 337.97345369,  328.34476819,  299.7938275 , ...,  316.35875591,
           320.30586092,   98.89789075],
         [ 317.43834413,  344.08285587,  363.85834553, ...,  326.40991628,
           333.05434526,   27.47038885],
         [ 345.94280758,  313.82961706,  329.6251119 , ...,  328.79858015,
           341.50743509,   35.18987289],
         ...,
         [ 350.9682166 ,  343.39073337,  330.40199869, ...,  320.77858198,
           327.60906872, -117.33154275],
         [ 337.37058397,  324.59889931,  316.23148189, ...,  331.02705983,
           354.45769453,  270.45637554],
         [ 308.85620359,  323.43895213,  335.575943  , ...,  359.56420462,
           314.23623739,   76.36487601]], unit=''), CCDData([[ 302.14286457,  316.36606655,  312.8689477 , ...,  340.94258463,
           359.66756426, -170.32414518],
         [ 322.51040167,  310.77055926,  307.38142157, ...,  344.43649943,
           322.3241425 , -113.54427393],
         [ 349.98807741,  308.79568645,  333.6

In [4]:
from skimage import data, draw
from skimage.registration import phase_cross_correlation
from scipy import ndimage

#Here we are alligning the second half of our master science V
scienceimage0 = astropy.nddata.CCDData.read('C:/Users/Andres/Desktop/NACHO/UNI/Year 3/Globular Clusters Project/Globular Clusters Data/20191111/science_V/20191111.00000055.fits', unit='adu')

scienceimagesVcorr = []
for i in range(len(scienceimagesV)):
    detected_shiftV = phase_cross_correlation(np.array(scienceimage0), np.array(scienceimagesV[i]), upsample_factor = 1000)
    print(detected_shiftV[0])
    scienceimagesV1 = scipy.ndimage.shift(scienceimagesV[i], detected_shiftV[0])
    scienceimagesV11 = astropy.nddata.CCDData(scienceimagesV1, unit = 'adu')
    scienceimagesVcorr.append(scienceimagesV11)

[-2.565  9.842]
[-2.939  9.707]
[-3.185 12.306]
[-3.005 11.165]
[-3.748 11.193]
[-4.117 11.52 ]
[-4.712 13.751]
[-4.926 13.078]
[-5.152 12.713]
[-5.497 14.34 ]
[-5.025 14.474]
[-4.913 13.132]
[-6.568 15.59 ]
[-6.492 14.231]
[-5.886 15.607]
[-6.306 16.56 ]
[-6.505 15.239]
[-6.403 17.937]
[-6.491 16.958]
[-7.021 17.704]


In [5]:
scienceimagesVcorr = astropy.nddata.CCDData(scienceimagesVcorr, unit='adu')
print(scienceimagesVcorr)

[[[  0.           0.           0.         ... 324.32609755 352.80445756
   329.05538828]
  [  0.           0.           0.         ... 338.96841279 335.96177918
   361.2103707 ]
  [  0.           0.           0.         ... 348.95385252 323.36788739
   363.90109844]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]]

 [[  0.           0.           0.         ... 363.30932711 331.69678001
   327.83625321]
  [  0.           0.           0.         ... 329.94719959 335.00105601
   306.88329654]
  [  0.           0.           0.         ... 335.82110783 295.5053209
   301.17684877]
  ...
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   0.           0.
     0.        ]
  [  0.           0.           0.         ...   

In [6]:
print((scienceimagesVcorr.shape))

(20, 4096, 4096)


In [7]:
#Finally we combine the final half of our master science V
ScienceV = ccdproc.combine((scienceimagesVcorr), method = 'median', format = 'fits')
hdu1 = CCDData.to_hdu(ScienceV)
hdu1.writeto('MasterScienceV2.fits', overwrite = True)

INFO: splitting each image into 2 chunks to limit memory usage to 16000000000.0 bytes. [ccdproc.combiner]
